In [6]:
import zipfile
import os

# Caminho da pasta
folder_path = r"C:\Users\Admin\Downloads\ecommerce\extracted_files"

# Listando todos os arquivos CSV
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

print("Arquivos CSV encontrados:")
for file in csv_files:
    print(file)



Arquivos CSV encontrados:
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv


In [16]:
import os
import pandas as pd

folder_path = r"C:\Users\Admin\Downloads\ecommerce\extracted_files"

csv_files = [
    "olist_customers_dataset.csv",
    "olist_geolocation_dataset.csv",
    "olist_orders_dataset.csv",
    "olist_order_items_dataset.csv",
    "olist_order_payments_dataset.csv",
    "olist_order_reviews_dataset.csv",
    "olist_products_dataset.csv",
    "olist_sellers_dataset.csv",
    "product_category_name_translation.csv"
]

dataframes = {}

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df_name = file.split(".")[0] 
    dataframes[df_name] = pd.read_csv(file_path)
    

for name, df in dataframes.items():
    print(f"\nAnálise de: {name}")
    print(f"Linhas e colunas: {df.shape}")
    print(f"Colunas: {list(df.columns)}")
    print(" ")
    print(f"Tipos de dados:\n{df.dtypes}\n")
    print(f"Dados ausentes:\n{df.isnull().sum()}\n")
    print(f"Primeiras 5 linhas:\n{df.head()}\n")
    print("-" * 50)


customers = dataframes["olist_customers_dataset"]





Análise de: olist_customers_dataset
Linhas e colunas: (99441, 5)
Colunas: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
 
Tipos de dados:
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

Dados ausentes:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Primeiras 5 linhas:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18

In [ ]:
import streamlit as st
import plotly.express as px

st.set_page_config(page_title="Dashboard de E-commerce", layout="wide")

@st.cache
def load_data(folder_path):
    csv_files = {
        "customers": "olist_customers_dataset.csv",
        "geolocation": "olist_geolocation_dataset.csv",
        "orders": "olist_orders_dataset.csv",
        "order_items": "olist_order_items_dataset.csv",
        "order_payments": "olist_order_payments_dataset.csv",
        "order_reviews": "olist_order_reviews_dataset.csv",
        "products": "olist_products_dataset.csv",
        "sellers": "olist_sellers_dataset.csv",
        "categories": "product_category_name_translation.csv",
    }
    dataframes = {}
    for key, file in csv_files.items():
        file_path = os.path.join(folder_path, file)
        dataframes[key] = pd.read_csv(file_path)
    return dataframes

data = load_data(folder_path)

customers = data["customers"]
geolocation = data["geolocation"]
orders = data["orders"]
order_items = data["order_items"]
order_payments = data["order_payments"]
order_reviews = data["order_reviews"]
products = data["products"]
sellers = data["sellers"]
categories = data["categories"]

# Títulos do dashboard
st.title("Dashboard de E-commerce 📊")
st.markdown("Análise dos dados da operação de um e-commerce brasileiro.")

col1, col2, col3 = st.columns(3)

col1.metric("Total de Clientes", customers["customer_id"].nunique())
col2.metric("Total de Pedidos", orders["order_id"].nunique())
col3.metric("Total de Produtos", products["product_id"].nunique())

# Gráfico 1: Pedidos por Status
st.subheader("Status dos Pedidos")
order_status_count = orders["order_status"].value_counts().reset_index()
order_status_count.columns = ["Status", "Quantidade"]
fig1 = px.bar(order_status_count, x="Status", y="Quantidade", title="Quantidade de Pedidos por Status", text="Quantidade")
st.plotly_chart(fig1, use_container_width=True)

# Gráfico 2: Distribuição de Pagamentos
st.subheader("Métodos de Pagamento")
payment_type_count = order_payments["payment_type"].value_counts().reset_index()
payment_type_count.columns = ["Tipo de Pagamento", "Quantidade"]
fig2 = px.pie(payment_type_count, values="Quantidade", names="Tipo de Pagamento", title="Distribuição de Tipos de Pagamento")
st.plotly_chart(fig2, use_container_width=True)

# Gráfico 3: Produtos mais vendidos
st.subheader("Produtos mais Vendidos")
product_sales = order_items["product_id"].value_counts().reset_index().head(10)
product_sales.columns = ["Produto ID", "Quantidade Vendida"]
fig3 = px.bar(product_sales, x="Produto ID", y="Quantidade Vendida", title="Top 10 Produtos mais Vendidos", text="Quantidade Vendida")
st.plotly_chart(fig3, use_container_width=True)

# Gráfico 4: Avaliações dos Pedidos
st.subheader("Distribuição de Avaliações")
review_score_count = order_reviews["review_score"].value_counts().reset_index()
review_score_count.columns = ["Nota", "Quantidade"]
fig4 = px.bar(review_score_count, x="Nota", y="Quantidade", title="Distribuição de Avaliações dos Pedidos", text="Quantidade")
st.plotly_chart(fig4, use_container_width=True)

# Filtro: Análise por Estado
st.subheader("Análise de Clientes por Estado")
customer_state_count = customers["customer_state"].value_counts().reset_index()
customer_state_count.columns = ["Estado", "Quantidade"]
fig5 = px.bar(customer_state_count, x="Estado", y="Quantidade", title="Clientes por Estado", text="Quantidade")
st.plotly_chart(fig5, use_container_width=True)

# Gráfico 6: Receita por Método de Pagamento
st.subheader("Receita por Tipo de Pagamento")
revenue_payment_type = order_payments.groupby("payment_type")["payment_value"].sum().reset_index()
revenue_payment_type.columns = ["Tipo de Pagamento", "Receita Total"]
fig6 = px.bar(revenue_payment_type, x="Tipo de Pagamento", y="Receita Total", title="Receita por Tipo de Pagamento", text="Receita Total")
st.plotly_chart(fig6, use_container_width=True)

st.markdown("---")